In [1]:
from sklearn.model_selection import GridSearchCV
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import time

import numpy as np
import pandas as pd

C:\Users\delpi\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\delpi\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Chargement des données

In [2]:
#torch.utils.data.DataLoader récupère lui même les classes des images
#Il suffit juste que les données 
train = torchvision.datasets.ImageFolder(root='C:\\Users\\delpi\\Mon Drive\\COURS\\AgroParisTech\\3A IODAA\\Fil rouge\\CIRSE\\training', transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True, num_workers=4)

test = torchvision.datasets.ImageFolder(root='C:\\Users\\delpi\\Mon Drive\\COURS\\AgroParisTech\\3A IODAA\\Fil rouge\\CIRSE\\test', transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True, num_workers=4)

# Création des modèles

Pour le moment, nous avons enlever juste le dernier neurone.
En fonction des résultats obtenus nous devrons probablement enlever quelques couches vers la fin.

## Fonction d'entrainement du modèle

In [3]:
def train(model, train_loader, optimizer, criterion):
    """
    Cette fonction permet d'entrainer un réseau de neuronne :
        -model : le réseau à entrainer
        -train_loader : le set d'entrainement (issus de torch.utils.data.DataLoader)
        -optimizer : choix de l'optimiseur
        -criterion : choix de la fonction de coût
    """
    model.train()
    for inputs, labels in train_loader:
        #réinitialisation du gradient
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [4]:
print(train_loader)

## Fonction pour valider le modèle

In [5]:
def validate(model, data, criterion):
    """
    Cette fonction permet de calculer la précision d'un modèle déja entrainé:
        -model : modèle pré-entrainé
        -data : set de donnée validation ou entrainement
        -criterion : la fonction de coût utilisé lors de l'apprentissage
    """
    model.eval()
    
    #Evaluation de l'accuracy pour chaque classe étant donnée qu'on est dans un cas déséquilibré
    nb_correct_predictions = [0] * len(data.dataset.classes)
    nb_total_predictions = [0] * len(data.dataset.classes)
    
    with torch.no_grad():
        for inputs, labels in data:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            for i in range(len(labels)):
                if labels[i] == predicted[i]:
                    nb_correct_predictions[labels[i]] += 1
                nb_total_predictions[labels[i]] += 1
                
    accuracy = [nb_correct_predictions[i] / nb_total_predictions[i] for i in range(len(data.dataset.classes))]
    return accuracy

## Liste des modèles

In [6]:
# 2 sorties pour absence et présence de cirses
# Attention, il existe plusieurs versions de Resnet
# Il faudra problablement en essayant d'autre
model = models.resnet18(pretrained=True)

# Réinitialiser la dernière couche pour adapter au nombre de classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)


C:\Users\delpi\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\delpi\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Fonction pour tester les différents paramètres

In [7]:
def perf_evaluation(model,data_train, data_test, num_epochs, early_stopping_threshold, learning_rate, param_grid, criterion, optimizer, scheduler):
    
    """
    Cette fonction permet pour un modèle donner de faire un grid_search afin de determiner le step_size et le gamma
    à partir d'une liste. La fonction renvoi le modèle avec les meilleurs paramètres ainsi qu'une table de données
    avec les statistiques de perofrmance du modèle.
        -model : modèle a entrainer
        -train : ze
        -test : 
        -num_epochs : le nombre d'epoques
        -early_stopping_threshold : le nombre d'époque minimum ou l'on n'observe aucune amèliariation des performances
        -learning_rate : 
        -param_grid : liste des step_size et gamma a tester
        -criterion : choix de la fonction de coût
        -optimizer : choix de l'optimiseur
        scheduler : 
    """

    counter = 0
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    epoch_list = []
    train_acc_list = []
    val_acc_list = []
    parameters_list = []
    time_list = []
    
    for epoch in range(num_epochs):
        
        start = time.time()
        train(model, data_train, optimizer, criterion)
        end = time.time()

        train_acc = validate(model, data_train, criterion)
        print("train acuracies :",train_acc)
        val_acc = validate(model, data_test, criterion)
        print("validation acuracies :",val_acc)
        
        epoch_list.append(epoch)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        parameters_list.append({'step_size': scheduler.step_size, 'gamma': scheduler.gamma})
        time_list.append(round((end-start) * 10**3))
        
        optimizer.step()
        scheduler.step()
        
        #Etant dans un cas désquilibré, nous voulons que tout les classes soient prédites avec précision
        #==> le produit de l'accuracy de l'ensemble des classes doit donc être le plus élevée possible
        #lors de l'évaluation du set de validation
        if val_acc[1]*val_acc[0] > best_acc:
            best_acc = val_acc[1]*val_acc[0]
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1

        if counter >= early_stopping_threshold:
            print("Early stopping at epoch: ", epoch+1)
            break
    
    return best_model_wts, epoch_list, train_acc_list, val_acc_list, parameters_list, time_list

## Définition des paramètres à tester (gridsearch)

In [8]:
num_epochs = 3000
early_stopping_threshold = 50
learning_rate = 0.001
param_grid = {'step_size': [5, 7, 10], 'gamma': [0.001, 0.01, 0.1, 0.5, 1, 10]}

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=10)
grid_search = GridSearchCV(scheduler, param_grid, cv=3)

# Analyse des modèles

In [ ]:
best_model_wts, epoch_list, train_acc_list, val_acc_list, parameters_list, time_list = perf_evaluation(model,train_loader, test_loader, num_epochs, early_stopping_threshold, learning_rate, param_grid, criterion, optimizer, scheduler)

train acuracies : [1.0, 0.012578616352201259]
validation acuracies : [1.0, 0.008547008547008548]
train acuracies : [0.9796672828096118, 0.8867924528301887]
validation acuracies : [0.9310689310689311, 0.717948717948718]
train acuracies : [1.0, 0.49056603773584906]
validation acuracies : [1.0, 0.2905982905982906]
train acuracies : [0.9930683918669131, 0.9119496855345912]
validation acuracies : [0.965034965034965, 0.6495726495726496]
train acuracies : [0.8174676524953789, 0.9779874213836478]
validation acuracies : [0.6573426573426573, 0.6324786324786325]
train acuracies : [1.0, 0.9308176100628931]


In [ ]:
perf = pd.DataFrame({'epoch' : epoch_list,
                     'train_accuracy' : train_acc_list,
                     'validation_accuracy' : val_acc_list,
                     'parameters' : parameters_list,
                     'time_ms': time_list})